<a href="https://colab.research.google.com/github/hasiburrahman1/NLP/blob/master/train_Embedding_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import tensorflow as tf

In [0]:
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
tf.__version__ 

'2.2.0'

In [0]:
data_raw = pd.read_csv(r"/content/drive/My Drive/Colab Notebooks/train.tsv",sep='\t',encoding = "ISO-8859-1")

In [0]:
print("Number of rows in data =",data_raw.shape[0])
print("Number of columns in data =",data_raw.shape[1])

Number of rows in data = 44933
Number of columns in data = 3


In [0]:
data_raw.head()raise

,label,question,answer_candidate
0,0,How many genes does the human hoxD cluster con...,"In comparison , HOX genes ( HOXA3 , HOXD9 , HO..."
1,0,How many genes does the human hoxD cluster con...,"We report here a novel NUP98 partner gene , HO..."
2,0,How many genes does the human hoxD cluster con...,"Heterozygous mutations of human HOXD13 , encod..."
3,0,How many genes does the human hoxD cluster con...,The apparently balanced de novo translocation ...
4,0,How many genes does the human hoxD cluster con...,Gene-gene interactions were identified between...


##### Preprocessing 

In [0]:
# Removing punctuations
data = data_raw.iloc[:,1:3]
data.replace("[^a-zA-Z]"," ",regex=True, inplace=True)

# Renaming column names for ease of access
list1 = [i for i in range(2)]
new_Index = [str(i) for i in list1]
data.columns = new_Index
data.head(5)

,0,1
0,How many genes does the human hoxD cluster con...,In comparison HOX genes HOXA HOXD HO...
1,How many genes does the human hoxD cluster con...,We report here a novel NUP partner gene HO...
2,How many genes does the human hoxD cluster con...,Heterozygous mutations of human HOXD encod...
3,How many genes does the human hoxD cluster con...,The apparently balanced de novo translocation ...
4,How many genes does the human hoxD cluster con...,Gene gene interactions were identified between...


In [0]:
' '.join(str(x) for x in data.iloc[0,0:3])

'How many genes does the human hoxD cluster contain  In comparison   HOX genes   HOXA    HOXD    HOXD     HOXD     and HOXD AS    a HOXD cluster antisense RNA       and WNT B expression were also significantly higher in sigmoid colon compared with the rectum  '

In [0]:
# Convertng headlines to lower case
for index in new_Index:
    data[index] = data[index].str.lower()
data.head(5)    

,0,1
0,how many genes does the human hoxd cluster con...,in comparison hox genes hoxa hoxd ho...
1,how many genes does the human hoxd cluster con...,we report here a novel nup partner gene ho...
2,how many genes does the human hoxd cluster con...,heterozygous mutations of human hoxd encod...
3,how many genes does the human hoxd cluster con...,the apparently balanced de novo translocation ...
4,how many genes does the human hoxd cluster con...,gene gene interactions were identified between...


In [0]:
' '.join(str(x) for x in data.iloc[0,0:3])

'how many genes does the human hoxd cluster contain  in comparison   hox genes   hoxa    hoxd    hoxd     hoxd     and hoxd as    a hoxd cluster antisense rna       and wnt b expression were also significantly higher in sigmoid colon compared with the rectum  '

In [0]:
corpus = []
for row in range(0,len(data.index)):
    corpus.append(' '.join(str(x) for x in data.iloc[row,0:3]))

In [0]:
corpus[0]

'how many genes does the human hoxd cluster contain  in comparison   hox genes   hoxa    hoxd    hoxd     hoxd     and hoxd as    a hoxd cluster antisense rna       and wnt b expression were also significantly higher in sigmoid colon compared with the rectum  '

In [0]:
corpus

['how many genes does the human hoxd cluster contain  in comparison   hox genes   hoxa    hoxd    hoxd     hoxd     and hoxd as    a hoxd cluster antisense rna       and wnt b expression were also significantly higher in sigmoid colon compared with the rectum  ',
 'how many genes does the human hoxd cluster contain  we report here a novel nup   partner gene   hoxd     not hoxd     in a pediatric patient with de novo aml having t              q     p       using a cdna panhandle pcr  ',
 'how many genes does the human hoxd cluster contain  heterozygous mutations of human hoxd     encoding polyalanine expansions or frameshifts   are believed to act by dominant negative or haploinsufficiency mechanisms and are predominantly associated with synpolydactyly phenotypes  ',
 'how many genes does the human hoxd cluster contain  the apparently balanced de novo translocation provides candidate loci for atypical and tcof  mutation negative cases of tcs  ',
 'how many genes does the human hoxd clus

#### Implement BAG OF WORDS

In [0]:
'''
from sklearn.feature_extraction.text import CountVectorizer

countvector=CountVectorizer(ngram_range=(2,2))
X=countvector.fit_transform(corpus)
'''

'\nfrom sklearn.feature_extraction.text import CountVectorizer\n\ncountvector=CountVectorizer(ngram_range=(2,2))\nX=countvector.fit_transform(corpus)\n'

#### TFidf Vectorizer

In [0]:
'''
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_v=TfidfVectorizer(max_features=10000,ngram_range=(1,3))
X=tfidf_v.fit_transform(corpus).toarray()
'''

'\nfrom sklearn.feature_extraction.text import TfidfVectorizer\n\ntfidf_v=TfidfVectorizer(max_features=10000,ngram_range=(1,3))\nX=tfidf_v.fit_transform(corpus).toarray()\n'

In [0]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

#### One hot representation

In [0]:
### Vocabulary size
voc_size=15000

In [0]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[5851,
  14090,
  13309,
  13224,
  2404,
  11315,
  12047,
  10750,
  1467,
  14376,
  58,
  919,
  13309,
  9574,
  12047,
  12047,
  12047,
  11596,
  12047,
  3321,
  14862,
  12047,
  10750,
  7057,
  4535,
  11596,
  9225,
  6303,
  6728,
  1491,
  10082,
  6236,
  3856,
  14376,
  10871,
  111,
  3138,
  8392,
  2404,
  427],
 [5851,
  14090,
  13309,
  13224,
  2404,
  11315,
  12047,
  10750,
  1467,
  4406,
  11479,
  6525,
  14862,
  3957,
  9624,
  7366,
  14408,
  12047,
  8312,
  12047,
  14376,
  14862,
  10750,
  6864,
  8392,
  12381,
  3265,
  904,
  11893,
  3739,
  4055,
  13954,
  11328,
  14862,
  3094,
  1082,
  11145],
 [5851,
  14090,
  13309,
  13224,
  2404,
  11315,
  12047,
  10750,
  1467,
  13434,
  13171,
  12752,
  11315,
  12047,
  10596,
  1895,
  11463,
  9365,
  1335,
  5924,
  6542,
  10047,
  4358,
  5806,
  12674,
  11338,
  9365,
  678,
  6549,
  11596,
  5924,
  11161,
  12931,
  8392,
  9772,
  1406],
 [5851,
  14090,
  13309,
  13224,
  2404

#### Embedding Representation

In [0]:
sent_length = len(max(corpus, key=len))
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[    0     0     0 ...  8392  2404   427]
 [    0     0     0 ...  3094  1082 11145]
 [    0     0     0 ...  8392  9772  1406]
 ...
 [    0     0     0 ...  5536 11671  7692]
 [    0     0     0 ... 12752  1803 10825]
 [    0     0     0 ... 10335 11671  7692]]


In [0]:
embedded_docs[0]

array([   0,    0,    0, ..., 8392, 2404,  427], dtype=int32)

### LSTM model

In [0]:
## Creating model

embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 4166, 40)          600000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 656,501
Trainable params: 656,501
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
## Creating Bidirectional LSTM model

embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4166, 40)          600000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 713,001
Trainable params: 713,001
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
## Creating LSTM with dropout ratio model

embedding_vector_features=40
model2=Sequential()
model2.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model2.add(Dropout(0.3))
model2.add(LSTM(100))
model2.add(Dropout(0.3))
model2.add(Dense(1,activation='sigmoid'))
model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4166, 40)          600000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 713,001
Trainable params: 713,001
Non-trainable params: 0
_________________________________________________________________
None


##### Get the Independent and Dependent Features

In [0]:
len(embedded_docs)

44933

In [0]:
y=pd.get_dummies(data_raw['label'])
y=y.iloc[:,1].values

In [0]:
X=np.array(embedded_docs)
y=np.array(y)

###### Train Test Split from sklearn

##### LSTM Model Training

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

In [0]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X,y),epochs=5,batch_size=128)

Epoch 1/5
236/236 [==============================] - 413s 2s/step - loss: 0.0544 - accuracy: 0.9873 - val_loss: 0.0523 - val_accuracy: 0.9878
Epoch 2/5
236/236 [==============================] - 413s 2s/step - loss: 0.0394 - accuracy: 0.9880 - val_loss: 0.0459 - val_accuracy: 0.9889
Epoch 3/5
236/236 [==============================] - 411s 2s/step - loss: 0.0278 - accuracy: 0.9909 - val_loss: 0.0429 - val_accuracy: 0.9923
Epoch 4/5
236/236 [==============================] - 418s 2s/step - loss: 0.0184 - accuracy: 0.9943 - val_loss: 0.0417 - val_accuracy: 0.9929
Epoch 5/5
236/236 [==============================] - 418s 2s/step - loss: 0.0112 - accuracy: 0.9963 - val_loss: 0.0445 - val_accuracy: 0.9925


##### Performance Metrics And Accuracy of LSTM model

In [0]:
y_pred1= np.argmax(model1.predict(X_test), axis=-1) 

In [0]:
## Import library to check accuracy
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score

In [0]:
matrix=confusion_matrix(y_test,y_pred1)
print(matrix)

[[14662     0]
 [  166     0]]


In [0]:
accuracy_score(y_test,y_pred1)

0.9888049635824117

In [0]:
report=classification_report(y_test,y_pred1)
print(report)

              precision    recall  f1-score   support

           0       0.99      1.00      0.99     14662
           1       0.00      0.00      0.00       166

    accuracy                           0.99     14828
   macro avg       0.49      0.50      0.50     14828
weighted avg       0.98      0.99      0.98     14828



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### RandomForest Classifier

In [0]:
## implement RandomForest Classifier
from sklearn.ensemble import RandomForestClassifier

In [0]:
randomclassifier=RandomForestClassifier(n_estimators=200,criterion='entropy')
randomclassifier.fit(X_train,y_train)

y_predict = randomclassifier.predict(X_test)

In [0]:
matrix=confusion_matrix(y_test,y_predict)
print(matrix)
score=accuracy_score(y_test,y_predict)
print(score)

[[14662     0]
 [  161     5]]
0.9891421634745077


In [0]:
report=classification_report(y_test,y_predict)
print(report)

              precision    recall  f1-score   support

           0       0.99      1.00      0.99     14662
           1       1.00      0.03      0.06       166

    accuracy                           0.99     14828
   macro avg       0.99      0.52      0.53     14828
weighted avg       0.99      0.99      0.98     14828

